In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import time
import yfinance as yf

import os
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


cf.go_offline()

from plotly.subplots import make_subplots

import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

In [25]:
PATH = "D:\Python\\"

#S_DATE = "2021-01-01"
#E_DATE = "2023-02-01"
#S_DATE_DT = pd.to_datetime(S_DATE)
#E_DATE_DT = pd.to_datetime(E_DATE)

In [26]:
#downloading the data using yfinance library
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)
    
    try:
        print("Get data for: ", ticker)
        df = stock.history(period = "5y")
        
        time.sleep(2)
        
        the_file = folder + ticker.replace(".", "_") + '.csv'
        print(the_file, " Saved")
        df.to_csv(the_file)
    except Exception as ex:
        print("Couldn't Get Data For : ", ticker)
        
save_to_csv_from_yahoo("D:\Python\\", "^NSEI")

Get data for:  ^NSEI
D:\Python\^NSEI.csv  Saved
Couldn't Get Data For :  ^NSEI


In [27]:
#functions for visualization

def get_stock_df_from_csv(ticker):
    
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col = 'Date', parse_dates = True)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df
#getting daily returns
def add_daily_returns(df):
    df['daily_return'] = (df['Close']/df['Close'].shift(1)) - 1
    df.fillna(0)
    return df

def add_cum_returns(df):
    df['cum_returns'] = (1 + df['daily_return']).cumprod()
    return df
#modifying the data for making bollinger bands
def bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96*df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96*df['Close'].rolling(window=20).std()
    return df
#modifying the data for making ichimoku bands
def ichimoku_bands(df):
    hv1 = df['High'].rolling(window=9).max()
    lv1 = df['Low'].rolling(window=9).min()
    df['conversion'] = (hv1 + lv1)/2
    
    hv2 = df['High'].rolling(window=26).max()
    lv2 = df['Low'].rolling(window=26).min()
    df['base'] = (hv2 + lv2)/2
    
    df['spanA'] = ((df['conversion'] + df['base'])/2).shift(26)
    hv3 = df['High'].rolling(window=52).max()
    lv3 = df['Low'].rolling(window=52).min()
    df['spanB'] = ((hv3 + lv3)/2).shift(26)
    df['lagging'] = df['Close'].shift(-26)
    
    return df

nifty_df = get_stock_df_from_csv('^NSEI')
add_daily_returns(nifty_df)
add_cum_returns(nifty_df)
bollinger_bands(nifty_df)
ichimoku_bands(nifty_df)
nifty_df.to_csv(PATH + 'NSEI.csv')

In [28]:
#this is the function for visualizing the bollinger bands, but for some reason, the candlestick plots were not showing
def plot_bollinger_bands(df, ticker):
    
    fig = go.Figure()
    
    candle = go.Candlestick(x = df.index, open=df['Open'], high=['High'],
                           low=df['Low'], close=df['Close'], name = 'CandleStick')
    upper_line = go.Scatter(x=df.index, y=df['upper_band'],
                           line = dict(color='rgba(0, 0, 250, 0.75)', width = 1),
                           name = "Upper Band")
    middle_line = go.Scatter(x =df.index, y=df['middle_band'],
                            line = dict(color='rgba(250, 0, 0, 0.75)', width = 0.75),
                            name = "Middle Band")
    lower_line = go.Scatter(x=df.index, y=df['lower_band'],
                           line = dict(color='rgba(0, 250, 0, 0.75)', width = 1),
                           name = "Lower Band")
    
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(middle_line)
    fig.add_trace(lower_line)
    
    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")
    fig.update_layout(title = ticker + " Bollinger Bands",
                     height = 600, width = 900, showlegend=True)
    fig.show()

In [29]:
#plot_bollinger_bands(nifty_df, "NSEI")
#import plotly.io
#plotly.io.renderers
#plot_bollinger_bands(nifty_df, "NIFTY")
#this is the code for printing Bollinger Bands without a specific function
fig = go.Figure()
candle = go.Candlestick(x=nifty_df.index, high=nifty_df.High, low=nifty_df.Low, open=nifty_df.Open, close=nifty_df.Close)
upper_line = go.Scatter(x=nifty_df.index, y=nifty_df['upper_band'],
                        line = dict(color='rgba(0, 0, 250, 0.75)', width = 1),
                        name = "Upper Band")
middle_line = go.Scatter(x =nifty_df.index, y=nifty_df['middle_band'],
                        line = dict(color='rgba(250, 0, 0, 0.75)', width = 0.75),
                        name = "Middle Band")
lower_line = go.Scatter(x=nifty_df.index, y=nifty_df['lower_band'],
                        line = dict(color='rgba(0, 250, 0, 0.75)', width = 1),
                        name = "Lower Band")
fig.add_trace(candle)
fig.add_trace(upper_line)
fig.add_trace(middle_line)
fig.add_trace(lower_line)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [30]:
#plotting the ichimoku curve
df = nifty_df.copy()

def get_fill_color(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'
    
fig = go.Figure()
candle = go.Candlestick(x=df.index, open=df.Open, close=df.Close, high=df.High, low=df.Low, name="Candle Stick")
df2 = df.copy()
df['label'] = np.where(df.spanA > df.spanB, 1, 0)
df['group'] = df.label.ne(df.label.shift()).cumsum()

df =df.groupby('group')
dfs = []

for name, data in df:
    dfs.append(data)
    
for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y=df.spanA, line=dict(color = 'rgba(0,0,0,0)')))
    fig.add_traces(go.Scatter(x=df.index, y=df.spanB, line=dict(color = 'rgba(0,0,0,0)'),
                   fill = 'tonexty', fillcolor = get_fill_color(df.label.iloc[0])))
    
baseline = go.Scatter(x=df2.index, y=df2.base, 
                      line=dict(color='pink', width = 2), name="Kijun-sen")
conversion = go.Scatter(x=df2.index, y=df2.conversion, 
                        line=dict(color='black', width = 1), name="Tenkan-sen")
spanA = go.Scatter(x=df2.index, y=df2.spanA,
                   line=dict(color='green', width = 2, dash='dot'), name="Senko Span A")
spanB = go.Scatter(x=df2.index, y=df2.spanB, 
                   line=dict(color='red', width = 1, dash='dot'), name="Senko Span B")
lag = go.Scatter(x=df2.index, y=df2.lagging, 
                 line=dict(color='purple', width = 2), name="Chiko Span")

fig.add_trace(candle)
fig.add_trace(baseline)
fig.add_trace(conversion)
fig.add_trace(lag)
fig.add_trace(spanA)
fig.add_trace(spanB)

fig.update_layout(height=1200, width=1800, showlegend=True)
fig.show()

In [31]:
#the portfolio made in Project_1
top_companies = ['KARURVYSYA',
 'DCMSHRIRAM',
 'MAZDOCK',
 'JKLAKSHMI',
 'RAINBOW',
 'FACT',
 'ADANIENT',
 'ADANIPOWER',
 'GESHIP',
 'ATGL',
 'AWL',
 'BLS',
 'KPITTECH',
 'TVSMOTOR',
 'METROBRAND',
 'TEJASNET',
 'SWANENERGY',
 'JKPAPER',
 'RVNL',
 'RAYMOND',
 'SUNTV']

shares = [20, 4, 0, 8, 8, 4, 3, 8, 5, 2, 12, 67, 14, 3, 10, 8, 10, 5, 81, 1, 0]

In [32]:
#Downloading the data of the portfolio using yfinance
for i in top_companies:
    save_to_csv_from_yahoo("D:\Python\Stocks_IND_Port\\", i + ".NS")

Get data for:  KARURVYSYA.NS
D:\Python\Stocks_IND_Port\KARURVYSYA_NS.csv  Saved
Get data for:  DCMSHRIRAM.NS
D:\Python\Stocks_IND_Port\DCMSHRIRAM_NS.csv  Saved
Get data for:  MAZDOCK.NS
D:\Python\Stocks_IND_Port\MAZDOCK_NS.csv  Saved
Get data for:  JKLAKSHMI.NS
D:\Python\Stocks_IND_Port\JKLAKSHMI_NS.csv  Saved
Get data for:  RAINBOW.NS
D:\Python\Stocks_IND_Port\RAINBOW_NS.csv  Saved
Get data for:  FACT.NS
D:\Python\Stocks_IND_Port\FACT_NS.csv  Saved
Get data for:  ADANIENT.NS
D:\Python\Stocks_IND_Port\ADANIENT_NS.csv  Saved
Get data for:  ADANIPOWER.NS
D:\Python\Stocks_IND_Port\ADANIPOWER_NS.csv  Saved
Get data for:  GESHIP.NS
D:\Python\Stocks_IND_Port\GESHIP_NS.csv  Saved
Get data for:  ATGL.NS
D:\Python\Stocks_IND_Port\ATGL_NS.csv  Saved
Get data for:  AWL.NS
D:\Python\Stocks_IND_Port\AWL_NS.csv  Saved
Get data for:  BLS.NS
D:\Python\Stocks_IND_Port\BLS_NS.csv  Saved
Get data for:  KPITTECH.NS
D:\Python\Stocks_IND_Port\KPITTECH_NS.csv  Saved
Get data for:  TVSMOTOR.NS
D:\Python\Stock

In [33]:
#making the portfolio data by multiplying all the price parameters of each stock by number of shares bought
def merge_df_all_columns(*tickers):
    mult_df = []
    for i in top_companies:
        try:
            df = pd.read_csv("D:\Python\Stocks_IND_Port\\" + i + "_NS.csv", index_col = "Date", parse_dates = True)
        except Exception as ex:
            print(ex)
        else:
            mult_df.append(df)
    df = mult_df[0]
    for k in df.columns:
        df[k] *= 0;
    for k in df.columns:
        for i in range(len(mult_df)):
            mult_df[i][k].fillna(0)
            df[k] += mult_df[i][k]*shares[i]
    return df

port_df = merge_df_all_columns(top_companies)
port_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-06-27 00:00:00+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-28 00:00:00+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-29 00:00:00+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-02 00:00:00+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-03 00:00:00+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-06-21 00:00:00+05:30,102045.915065,104380.454576,100941.742259,102230.100693,1.898487e+09,0.0,0.0
2023-06-22 00:00:00+05:30,102567.149469,103995.383174,100540.816923,101983.798355,1.218786e+09,24.0,0.0
2023-06-23 00:00:00+05:30,101961.700157,102816.649864,98472.550774,100008.349739,8.170184e+08,3.0,0.0


In [34]:
#adding the parameters for visualization
add_daily_returns(port_df)
add_cum_returns(port_df)
bollinger_bands(port_df)
ichimoku_bands(port_df)
df.to_csv("D:\Python\\My_Portfolio.csv" )

In [35]:
#Plotting the Bollinger Bands
fig = go.Figure()
candle = go.Candlestick(x=port_df.index, high=port_df.High, low=port_df.Low, open=port_df.Open, close=port_df.Close)
upper_line = go.Scatter(x=port_df.index, y=port_df['upper_band'],
                        line = dict(color='rgba(0, 0, 250, 0.75)', width = 1),
                        name = "Upper Band")
middle_line = go.Scatter(x =port_df.index, y=port_df['middle_band'],
                        line = dict(color='rgba(250, 0, 0, 0.75)', width = 0.75),
                        name = "Middle Band")
lower_line = go.Scatter(x=port_df.index, y=port_df['lower_band'],
                        line = dict(color='rgba(0, 250, 0, 0.75)', width = 1),
                        name = "Lower Band")
fig.add_trace(candle)
fig.add_trace(upper_line)
fig.add_trace(middle_line)
fig.add_trace(lower_line)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [36]:
#Plotting the Ichimoku Kinko Hyo
df = port_df.copy()
fig = go.Figure()
candle = go.Candlestick(x=df.index, open=df.Open, close=df.Close, high=df.High, low=df.Low, name="Candle Stick")
df2 = df.copy()
df['label'] = np.where(df.spanA > df.spanB, 1, 0)
df['group'] = df.label.ne(df.label.shift()).cumsum()

df =df.groupby('group')
dfs = []

for name, data in df:
    dfs.append(data)
    
for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y=df.spanA, line=dict(color = 'rgba(0,0,0,0)')))
    fig.add_traces(go.Scatter(x=df.index, y=df.spanB, line=dict(color = 'rgba(0,0,0,0)'),
                   fill = 'tonexty', fillcolor = get_fill_color(df.label.iloc[0])))
    
baseline = go.Scatter(x=df2.index, y=df2.base, 
                      line=dict(color='pink', width = 2), name="Kijun-sen")
conversion = go.Scatter(x=df2.index, y=df2.conversion, 
                        line=dict(color='black', width = 1), name="Tenkan-sen")
spanA = go.Scatter(x=df2.index, y=df2.spanA,
                   line=dict(color='green', width = 2, dash='dot'), name="Senko Span A")
spanB = go.Scatter(x=df2.index, y=df2.spanB, 
                   line=dict(color='red', width = 1, dash='dot'), name="Senko Span B")
lag = go.Scatter(x=df2.index, y=df2.lagging, 
                 line=dict(color='purple', width = 2), name="Chikou Span")

fig.add_trace(candle)
fig.add_trace(baseline)
fig.add_trace(conversion)
fig.add_trace(lag)
fig.add_trace(spanA)
fig.add_trace(spanB)

fig.update_layout(height=1200, width=1800, showlegend=True)
fig.show()